# Explicación Reto1:

1. Encuentra una base de datos con al menos tres variables (puedes ser más de tres) en la página de datos abiertos del gobierno colombiano. La base de datos debe ser de tu área de desempeño. Una de las  variables debe ser la variable a predecir, las adicionales serán las predictoras. En este caso las variables no debe tener menos de 45 observaciones.
2. Hacer la primera revisión de los datos y determinar si se requiere o no hacer tratamiento de datos nulos o atípicos.
3. Realizar, para cada modelo, la predicción sin preprocesamiento y con preprocesamiento.
4. Debes implementar diferentes modelos de regresión con las siguientes variaciones:
- Modelo de regresión múltiple.
- Modelo de regresión con regularización Lasso  encontrando el valor de alpha para el mejor desempeño.
- Modelo de regresión con regularización Ridge encontrando el valor de alpha para el mejor desempeño

 Si se tienen pocos datos se debe hacer validación cruzada. Si se tienen muchos datos se debe hacer partición en entrenamiento y validación 80/20.
Reportar para todos los casos el score del test.

In [31]:
#Importación de las librerías
from sklearn import datasets #Probar y practicar algoritmos
import numpy as np #Operaciones matemáticas
import pandas as pd #Herramienta de manipulación y análisis
import matplotlib.pyplot as plt #Creación de gráficas
from sklearn.linear_model import LinearRegression #Regresión lineal
from sklearn.preprocessing import PolynomialFeatures #Regresón polinómica
from sklearn.pipeline import Pipeline #Encadenar pasos de procesamiento y modelado en un solo obajeto
from sklearn.model_selection import cross_val_score #Validación cruzada para evalualr la precisión del modelo
from sklearn.linear_model import Ridge #Implementa la regresión Ridge
from sklearn.metrics import mean_squared_error #Error cuadrático medio
from sklearn.model_selection import train_test_split #Divide un conjuento de datos en subconjuntos de entrenamiento y prueba para evaluar el modelo
from sklearn.model_selection import GridSearchCV #Realiza una búsqueda sobre los parámetros especidicados para un estimador, utilizando validación cruzada para encontrar la mejor combinación.
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, StandardScaler #Escala las características de los datos a un rango específico.
from sklearn.preprocessing import StandardScaler #Estandariza las características eliminando la media y escalando a la varianza unitaria.
import torch #utilizada para construir y entrenar modelos de aprendizaje profundo.

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Base de Datos:
De la página de datos abiertos del gobierno de Colombia, para este ejercicio tomaremos la base de datos `Ejecución De Ingresos Y Gastos Por Municipio Del Valle Del Cauca`

In [33]:
#Cargamos los datos del archivo CSV
datos = pd.read_excel("/content/drive/MyDrive/Aprendizaje Supervisado/Bases de Datos/Ejecuci_n_De_Ingresos_Y_Gastos_Por_Municipio_Del_Valle_Del_Cauca_20240906.xlsx")
datos.head()

,Orden,Código Depto,Código Municipio,Municipio,Latitud,Longitud,Nombre Unidad Ejecutora,Presupuesto Inicial (Millones),Presupuesto Definitivo (Millones),Total Compromisos (Millones),Total Obligaciones (Millones),Pagos Efectivos a Fecha de Corte (Millones),Compromisos / Presupuesto Definitivo,Pagos / Presupuesto Definitivo,Pagos / Compromisos,Año
0,3620,76,76001,CALI,342158,-765205.0,ADMINISTRACIÓN CENTRAL,2638090,2583770,2443910,2436000,1922330,0.95,0.74,0.79,2020
1,3621,76,76001,CALI,342158,-765205.0,CONCEJO - SOLO MUNICIPIOS,87257,8736,8507,8507,8476,0.97,0.97,1.00,2020
2,3622,76,76001,CALI,342158,-765205.0,CONTRALORIA,119455,12094,12091,12091,10251,1.00,0.85,0.85,2020
3,3623,76,76001,CALI,342158,-765205.0,PERSONERIA - SOLO MUNICIPIOS,85801,8733,8733,8733,7949,1.00,0.91,0.91,2020
4,3624,76,76001,CALI,342158,-765205.0,SECRETARÍA DE EDUCACIÓN,95,9400,5000,5000,5000,0.53,0.53,1.00,2020


In [35]:
#Eliminaremos las columnas que no son de nuestro interes para este ejercicio.
#(Orden, Código Depto, Código Municipio, Latitud, Longitud)
datos = datos.drop(columns=['Orden', 'Código Depto', 'Código Municipio', 'Latitud', 'Longitud'])
datos.head()

,Municipio,Nombre Unidad Ejecutora,Presupuesto Inicial (Millones),Presupuesto Definitivo (Millones),Total Compromisos (Millones),Total Obligaciones (Millones),Pagos Efectivos a Fecha de Corte (Millones),Compromisos / Presupuesto Definitivo,Pagos / Presupuesto Definitivo,Pagos / Compromisos,Año
0,CALI,ADMINISTRACIÓN CENTRAL,2638090,2583770,2443910,2436000,1922330,0.95,0.74,0.79,2020
1,CALI,CONCEJO - SOLO MUNICIPIOS,87257,8736,8507,8507,8476,0.97,0.97,1.00,2020
2,CALI,CONTRALORIA,119455,12094,12091,12091,10251,1.00,0.85,0.85,2020
3,CALI,PERSONERIA - SOLO MUNICIPIOS,85801,8733,8733,8733,7949,1.00,0.91,0.91,2020
4,CALI,SECRETARÍA DE EDUCACIÓN,95,9400,5000,5000,5000,0.53,0.53,1.00,2020


## 2. Revisión de Datos nulos y atípicos:


In [36]:
#Datos faltantes por columnas
datos.isnull().sum()

,0
Municipio,0
Nombre Unidad Ejecutora,0
Presupuesto Inicial (Millones),0
Presupuesto Definitivo (Millones),0
Total Compromisos (Millones),0
Total Obligaciones (Millones),0
Pagos Efectivos a Fecha de Corte (Millones),0
Compromisos / Presupuesto Definitivo,0
Pagos / Presupuesto Definitivo,0
Pagos / Compromisos,0


In [37]:
#Resumen estadístico de las columnas
datos.describe()

,Presupuesto Inicial (Millones),Presupuesto Definitivo (Millones),Total Compromisos (Millones),Total Obligaciones (Millones),Pagos Efectivos a Fecha de Corte (Millones),Compromisos / Presupuesto Definitivo,Pagos / Presupuesto Definitivo,Pagos / Compromisos,Año
count,1.490000e+02,1.490000e+02,1.490000e+02,1.490000e+02,1.490000e+02,149.000000,149.000000,149.000000,149.0
mean,4.147863e+04,4.528295e+04,4.290014e+04,4.284303e+04,3.945683e+04,0.927785,0.896510,0.966779,2020.0
std,2.266613e+05,2.119674e+05,2.005250e+05,1.998855e+05,1.584574e+05,0.101041,0.109241,0.054491,0.0
min,9.500000e+01,1.010000e+02,1.020000e+02,1.020000e+02,1.020000e+02,0.260000,0.250000,0.670000,2020.0
25%,6.780000e+02,9.150000e+02,8.370000e+02,8.370000e+02,8.830000e+02,0.920000,0.870000,0.950000,2020.0
50%,2.438000e+03,3.783000e+03,3.417000e+03,3.417000e+03,3.760000e+03,0.960000,0.920000,1.000000,2020.0
75%,1.497000e+04,6.570000e+04,6.280000e+04,6.280000e+04,6.120000e+04,0.990000,0.970000,1.000000,2020.0
max,2.638090e+06,2.583770e+06,2.443910e+06,2.436000e+06,1.922330e+06,1.000000,1.000000,1.000000,2020.0


La base de datos no tiene valores faltantes. Los valores financieros no parecen tener datos atípicos de mayor relevancia.

##3. Realización de cada modelo:

Predicción sin preprocesamiento y con preprocesamiento.

### Definición de las variables:
- **Variable a predecir:** Presupuesto Definitivo (Millones)
- **Variables predictorias:**
  - Presupuesto Inicial (Millones)
  - Total Compromisos (Millones)
  - Total Obligaciones (Millones)


In [38]:
#Seleccionamos las variables predictorias y la variable a predecir o objetivo
X = datos[['Presupuesto Inicial (Millones)', 'Total Compromisos (Millones)', 'Total Obligaciones (Millones)']]
y = datos['Presupuesto Definitivo (Millones)']

In [39]:
#Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Regresión Múltiple

#### Sin preprocesamiento

In [40]:
#Creamos y entrenamos el modelo de regresión múltiple
modelo_multiple = LinearRegression()
modelo_multiple.fit(X_train, y_train)

LinearRegression()

In [41]:
#Predecir y calcular el score del conjunto de prueba
y_pred_multiple = modelo_multiple.predict(X_test)
score_multiple = modelo_multiple.score(X_test, y_test)
print("Score del modelo de regresión múltiple:", score_multiple)

Score del modelo de regresión múltiple: 0.9921497304072925


Este score indica que el modelo es muy preciso (99.21%) para el conjunto de datos.

#### Con prepocesamiento

In [69]:
#vamos a agregar una nueva característica con el fin de agregar un grado más al modelo
poly = PolynomialFeatures(degree = 5, include_bias = True)
X_p = poly.fit_transform(X)

#vamos a revisar el tamaño de X_p
X_p.shape

(149, 56)

In [70]:
#Entrenamos el modelo
model.fit(X_p, y)
#Calculamos el score
model.score(X_p, y)

0.9976268097126063

In [48]:
#Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Creamos y entrenamos el modelo con los datos escalados
modelo_lineal = LinearRegression() # define modelo_lineal as LinearRegression object
modelo_lineal.fit(X_train_scaled, y_train)

#Predecir y calcular el score del conjunto de prueba
y_pred_scaled = modelo_lineal.predict(X_test_scaled)
score_scaled = modelo_lineal.score(X_test_scaled, y_test)

print("Score del modelo de regresión múltiple con preprocesamiento:", score_scaled)

Score del modelo de regresión múltiple con preprocesamiento: 0.9921497304072918


Al ser el score casi el mismo que sin preprocesamiento quiere decir que el escalado no tuvo un impacto significativo en el rendimiento del modelo.

## Regresión con regularización Lasso

In [49]:
#Para ignorar los warnings
import warnings
warnings.filterwarnings("ignore")

In [60]:
from operator import mod
#Importamos la lista de coeficientes
coeficientes = []
train_errors = []
test_errors = []

#Definifmos los posbiles valores que puede tomar lambda (alpha)
alphas = [0, 0.01, 0.1, 1, 10, 100]

from sklearn.linear_model import Lasso
#Creamos el ciclo para la sintonización de lambda
for alpha in alphas:
    #Creamos el modelo
    modelo_lasso = Lasso(alpha = alpha)
    #Entrenamos el modelo
    modelo_lasso.fit(X_train, y_train)
    #Guardar los coeficientes # Moved this line inside the loop
    coeficientes.append(modelo_lasso.coef_)

    #Calcular el error en el conjunto de entrenamiento y prueba
    train_error = (mean_squared_error(y_train, modelo_lasso.predict(X_train)))
    test_error = (mean_squared_error(y_test, modelo_lasso.predict(X_test)))

    #Guardamos los errores
    train_errors.append(train_error)
    test_errors.append(test_error)

# Mostrar los coeficientes y errores para cada valor de alpha
for i, alpha in enumerate(alphas):
    print(f"Alpha: {alpha}")
    print(f"Coeficientes: {coeficientes[i]}")
    print(f"Error cuadrático medio para el conjunto de entrenamiento: {train_errors[i]}")
    print(f"Error cuadrático medio para el conjunto de prueba: {test_errors[i]}\n")

Alpha: 0
Coeficientes: [ 0.01586599  1.04184843 -0.00287946]
Error cuadrático medio para el conjunto de entrenamiento: 134505119.74138862
Error cuadrático medio para el conjunto de prueba: 3598142.313936091

Alpha: 0.01
Coeficientes: [ 0.01586599  1.04184843 -0.00287946]
Error cuadrático medio para el conjunto de entrenamiento: 134505119.7415026
Error cuadrático medio para el conjunto de prueba: 3598142.313635679

Alpha: 0.1
Coeficientes: [ 0.01586599  1.04184843 -0.00287946]
Error cuadrático medio para el conjunto de entrenamiento: 134505119.7425279
Error cuadrático medio para el conjunto de prueba: 3598142.310931944

Alpha: 1
Coeficientes: [ 0.01586598  1.0418484  -0.00287943]
Error cuadrático medio para el conjunto de entrenamiento: 134505119.75278133
Error cuadrático medio para el conjunto de prueba: 3598142.2838947424

Alpha: 10
Coeficientes: [ 0.01586598  1.04184811 -0.00287914]
Error cuadrático medio para el conjunto de entrenamiento: 134505119.85531595
Error cuadrático medio pa

In [64]:
# Entrenar el modelo Lasso con un valor de alpha
lasso = Lasso(alpha=10)  # Ajusta el valor de alpha según sea necesario
lasso.fit(X_train_scaled, y_train)

# Predecir en el conjunto de prueba
y_pred = lasso.predict(X_test_scaled)

# Evaluar el rendimiento
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse}")
print(f"Coeficientes del modelo: {lasso.coef_}")

Error Cuadrático Medio (MSE): 3660100.1315768734
Coeficientes del modelo: [4.13727093e+03 2.31187304e+05 1.18787963e+02]


El valor de alpha = 10 logró reducir ligeramente el MSE, lo que indica que el modelo es menos propenso al sobreajuste.

Los coeficientes aún no se han reducido a cero, lo que significa que todas las variables siguen siendo relevantes para la predicción.

##  Regresión con regularización Ridge

In [72]:
#Instanciamos el modelo
model = Ridge()

#Definir un rango de valores para alpha
alpha = np.logspace(-4,2)

#Creamos un diccionario con todos los hiperparámetros a sintonizar
grid = dict(alpha = alpha)

#Importamos librerías adicionales
from sklearn.model_selection import GridSearchCV #Para la búsqueda de hiperparámetros

#Generamos la búsqueda por grip o exhaustiva
grid_search = GridSearchCV(estimator = model, param_grid = grid, n_jobs=1, cv = 5,
                           scoring = "neg_mean_squared_error")

#Generaamos la búsqueda con el conjunto de entrenamiento
grid_result = grid_search.fit(X_train, y_train)

In [73]:
#Extraemos el valor del hiperparámetro sintonizado
best_alpha = grid_result.best_params_["alpha"]

#Visualizamos la información
print(f"El mejor RMSE fue de {-grid_result.best_score_:.2f}, con una lambda de {best_alpha}")

El mejor RMSE fue de 200730172.90, con una lambda de 0.0001


In [75]:
#Ahora evaluamos el modelo con el conjunto de test o los datos de prueba

#Tomamos el mejor modelo de la búsqueda
best_model = grid_search.best_estimator_

#Transformamos los datos de prueba a las características polinómicas
# Se usa X_test en lugar de X_test_p
y_pred = best_model.predict(X_test)

#Calculamos el RMSE
RMSE = mean_squared_error(y_test, y_pred)

#Visualizamos el resultado
print(f"El RMSE para el conjunto de prueba es {RMSE:.2f}")

El RMSE para el conjunto de prueba es 7810917.88


El RMSE de entrenamiento es demasiado bajo, lo que indica que el modelo no necesita una regularización. Esto sugiere que las variables predictorias ya tienen una buena relación con la variable objetivo.

El RMSE en el conjunto de prueba es mayor que en el conjunto de entrenamiento, pero esto es normal y esperado. La diferencia no es demasiado grande, lo que indica que el modelo está generalizando bien a los nuevos datos y no está sobreajustado a los datos de entrenamiento.